In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import re
import numpy.ma as ma

from hbmep.config import Config
from hbmep.model.utils import Site as site

from models import NonHierarchicalBayesianModel
from constants import (
    DATA_PATH,
    TOML_PATH
)

In [5]:
config = Config(toml_path=TOML_PATH)
model = NonHierarchicalBayesianModel(config=config)
df = pd.read_csv("/home/andres/data/hbmep-processed/J_RCML_000/data.csv")

In [10]:
combinations = pickle.load(open('/home/andres/repos/rat-mapping-paper/reports/J_RCML_000/combine.pkl', 'rb'))

In [18]:
df['compound_position'].unique()

array(['C5M-C5L', 'C5M-C6L', 'C7L-C8L', '-C5M', 'C7M-C8L', 'C8M-C8L',
       '-C8M', '-C6L', '-C7M', '-C8L', 'C7M-C8M', 'C5L-C6L', '-C5L',
       'C6L-C7L', '-C7L', 'C7M-C7L', 'C6M-C7M', 'C5M-C6M', '-C6M',
       'C6M-C7L', 'C6M-C6L'], dtype=object)

In [30]:
subjects = df["participant"].unique().tolist()
positions = ['L', 'M']
# levels = ['C6-C7', 'C7-C8', 'C7-C6', 'C8-C7']
levels = ['C5-C6', 'C6-C7', 'C7-C8']
# positions = [p for p in positions if (p.split('-')[1][2] in targets and p.split('-')[1][1] != "7")]
muscles = model.response


In [31]:
a = []
mask = []
for subject in subjects:
    for position in positions:
        for level in levels:
            for muscle in muscles:
                l, r = level.split('-')
                pos = l + position + '-' + r + position
                key = (subject, pos, muscle)
                if key in combinations.keys():
                    temp = combinations[key]['a']
                    temp = temp.reshape(-1, 1)
                    a.append(temp)
                    mask.append(np.full((4000,1), False))  
                else: 
                    mask.append(np.full((4000,1), True))
                    a.append(np.zeros(4000).reshape(-1, 1))
arr = np.array(a)
arr = arr.reshape(len(subjects), len(positions),len(levels), len(muscles), *arr.shape[1:])
mask = np.array(mask)
mask = mask.reshape(len(subjects), len(positions),len(levels),len(muscles), *mask.shape[1:])

arr = ma.masked_array(arr, mask=mask)
arr = arr[..., 0]

In [32]:
arr.shape #subs, positions, levels, muscles, posterior_samples

(8, 2, 3, 6, 4000)

In [34]:
muscles

['LADM', 'LBiceps', 'LDeltoid', 'LECR', 'LFCR', 'LTriceps']

In [39]:
arr.shape

(8, 2, 3, 6, 4000)

In [42]:
l_samples = arr[..., [0], :]
l_samples.shape

(8, 2, 3, 1, 4000)

In [40]:
l_samples = arr[:, 0, ..., [0], :] 
l_samples.shape

(1, 8, 3, 4000)

In [57]:
muscle_ind = 3

l_samples = arr[..., [muscle_ind], :] 
m_samples = arr[..., [muscle_ind], :]

l_samples = l_samples[:, 0, ...] 
m_samples = m_samples[:, 1, ...] 

t = l_samples.mean(axis=-1) - m_samples.mean(axis=-1)
ttest = stats.ttest_1samp(
    t, popmean=0, alternative="greater", axis=(0, 1, 2)
)
ttest

TtestResult(statistic=-0.7993128364071947, pvalue=0.7824321689827636, df=17)